--- In this example we look at the properties of one of the sources identified by segmentation.

In [1]:
import numpy as np
from astropy.io import fits

In [2]:
image_dir = 'data' # define image directory relative to this script

In [3]:
f = 'f125w' # filter

In [4]:
sci = fits.getdata(f'{image_dir}/{f}_sci.fits') # read science FITS file data into numpy array
wht = fits.getdata(f'{image_dir}/{f}_wht.fits') # read weight FITS file data into numpy array

In [5]:
mask = fits.getdata(f'{image_dir}/mask.fits') # read in the image mask
sci = np.ma.masked_array(sci, mask = mask) # apply the mask to our science image
wht = np.ma.masked_array(wht, mask = mask) # apply the mask to our weight image

--- cut out a portion of the image for analysis

In [6]:
x = 2500 # pixel x-centre of cutout, must be an integer
y = 2500 # pixel y-centre of cutout, must be an integer
r = 100 # width/2 of cutout, must be int

In [7]:
sci = sci[x-r:x+r, y-r:y+r] # cutout a portion of the science image
wht = wht[x-r:x+r, y-r:y+r] # cutout a portion of the weight image

--- define the noise in each pixel and make a significance map (signal/noise)

In [8]:
noise = 1./np.sqrt(wht) # conversion from weight to noise
sig = sci/noise # signifance map

--- now run segmentation on the image.

In [9]:
from photutils import detect_sources
import matplotlib.pyplot as plt

In [10]:
threshold = 2.5 # require each pixel have a significance of >2.5 (since we're using the significance image)
npixels = 5 # require at least 5 connected pixels

In [11]:
segm = detect_sources(sig, threshold, npixels=npixels) # make segmentation image

In [12]:
i = 11

--- lets again determine the total flux of that same source by simply summing the pixels on the orginal science image where the segmentation map = the index of our target galaxy:

In [13]:
signal = np.sum(sci[np.where(segm.data==i)])

In [14]:
print(f'the signal is: {signal}')

the signal is: 1.711333990097046


--- the signal alone isn't very useful, we need an estimate of the uncertainty or error. The error is the sqrt(sum(noise_i**2))

In [15]:
error = np.sqrt(np.sum(noise[np.where(segm.data==i)]**2))

In [16]:
print(f'the error (noise) is: {error}')
print(f'the signal-to-noise is: {signal/error}')

the error (noise) is: 0.016436539590358734
the signal-to-noise is: 104.11766052246094
